Загружаем библиотеки

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Загружаем исходный датасет

In [ ]:
df = pd.read_csv("dataset.csv")
df.sample(10)

In [ ]:
df.info()

Видно, что есть пропущенные данные, их нужно удалить

In [ ]:
df = df.dropna()
df.shape

Так же удаляем ненужную колонку "Unnamed: 0"

In [ ]:
df = df.drop(columns=["Unnamed: 0"])
df.columns

Смотрим, есть ли повторяющиеся значения в statement

In [ ]:
df.describe()

Они есть, стоит оставить только уникалбные значения в statement

In [ ]:
df = df.drop_duplicates(subset=["statement"])
df.describe()

Чистим текст от ссылок, переносов строки и тд, а также переносим к нижнему регистру

In [ ]:
import re

def pre_process(text):
    text = re.sub(r'http[s]?://\S+', '', text)
    # text = re.sub(r'\[.*?\]\(.*?\)', '', text)
    text = re.sub(r'@\w+', '', text)
    # text = re.sub(r'[^\w\s]', '', text)
    return text.strip().lower()

df['statement'] = df['statement'].apply(pre_process)
df.sample(10)

Делим выборку на трейн и тест

In [ ]:
k = 0.2
classes = df["status"].unique()
test = pd.DataFrame(columns=df.columns)
for lable in classes:
    temp = df[df["status"] == lable]
    temp = temp.sample(int(temp.shape[0] * k))
    test = pd.concat([test, temp], ignore_index=True)
    df = df.drop(index=temp.index)
print("-------- train ----------")
print(test["status"].value_counts())
print("\n\n-------- test ----------")
print(df["status"].value_counts())

Теперь посмотрим на колличество семплов для каждого класса

In [ ]:
plt.bar(df["status"].value_counts().keys(), df["status"].value_counts().values)
plt.xticks(fontsize=6)
plt.show()

На лицо дисбаланс классов, сделаем ансемплинг частых классов и оверсемплинг редких, чтобы их количество стало равным 6000

Делаем ансемплинг

In [ ]:
num = 6000
# ["Anxiety", "Bipolar", "Stress"]
for lable in ["Normal", "Depression", "Suicidal"]:
    unsampled = df[df["status"] == lable].sample(num)
    df = df.drop(index = df[df["status"] == lable].index)
    df = pd.concat([df, unsampled], ignore_index=True)

plt.bar(df["status"].value_counts().keys(), df["status"].value_counts().values)
plt.xticks(fontsize=6)
plt.show()


Делаем оверсемплинг

In [16]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=7)
X_train_resampled, y_train_resampled = ros.fit_resample(df.drop(columns=["status"]).values.reshape(-1, 1), df["status"])

In [ ]:
y_train_resampled.value_counts()

Делаем эмбеденги

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

В массиве X_train_resampled каждый элемент - это массив с одной строкой. Для модели нужен массив строк.

In [18]:
X_train_resampled = X_train_resampled.tolist()

for i in range(len(X_train_resampled)):
    X_train_resampled[i] = X_train_resampled[i][0]

In [16]:
embeddings = model.encode(X_train_resampled)

Сохраняем эмбединги тренеровочной выборки в json

In [ ]:
import json

with open("train_embeddings_8000.json", "w") as f:
    json.dump(embeddings.tolist(), f)


Сохраняем таргеты тренеровочной выборки в json

In [21]:
with open("train_target_2.json", "w") as f:
    json.dump(y_train_resampled.tolist(), f)

Считаем эмбединги тестовой выборки

In [22]:
embeddings = model.encode(test["statement"])

Сохраняем эмбединги тестовой выборки в json

In [24]:
with open("test_embeddings_8000.json", "w") as f:
    json.dump(embeddings.tolist(), f)

Сохраняем таргеты тестовой выборки в json

In [26]:
with open("test_target_8000.json", "w") as f:
    json.dump(test_target.tolist(), f)